# Preparing data from .Json

In [392]:
from pymongo import MongoClient
import pandas as pd
import requests
import re
from lib import *
from pandas.io.json import json_normalize
import os 
from dotenv import load_dotenv
load_dotenv()
import json

In [393]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

**I am going to filter by "category_code" looking for those companies which are related with gaming industry and my worker´s interests (design) #Find a detailed explanation in the README.md**

In [3]:
all_categories = db.companies.find({},{"category_code":1,"name":1, "_id" : 0}) #Checking categories
categories_data = pd.DataFrame(all_categories)
set(categories_data.category_code)   

{None,
 'advertising',
 'analytics',
 'automotive',
 'biotech',
 'cleantech',
 'consulting',
 'design',
 'ecommerce',
 'education',
 'enterprise',
 'fashion',
 'finance',
 'games_video',
 'government',
 'hardware',
 'health',
 'hospitality',
 'legal',
 'local',
 'manufacturing',
 'medical',
 'messaging',
 'mobile',
 'music',
 'nanotech',
 'network_hosting',
 'news',
 'nonprofit',
 'other',
 'photo_video',
 'public_relations',
 'real_estate',
 'search',
 'security',
 'semiconductor',
 'social',
 'software',
 'sports',
 'transportation',
 'travel',
 'web'}

# Fulfilling developers´s preferens

In [4]:
#Games_video because it is always a good idea to know where your competence is.
#The other 3 in order to fulfill developers preferences (It seems that are startups - raised at least $ 1M)
query= {"$or":[   
                {"category_code":"games_video"},
                {"category_code":"software"},
                {"category_code":"mobile"},
                {"category_code":"social"},
                {"category_code":"network_hosting"},
                {"category_code":"ecommerce"}
                ],
                "total_money_raised":{"$regex":"(\$\d+\M)|(\$\d+B)"}                  
             }
data=db.companies.find(query,{"offices":1,"name": 1,"category_code":1,"total_money_raised":1,"_id":0})
data=list(data)
len(data)

643

In [5]:
#As a DataFrame
techdata = pd.DataFrame(data)
techdata = techdata.explode("offices")
techdata

,name,category_code,total_money_raised,offices
0,Joost,games_video,$45M,"{'description': '', 'address1': '100 5th Ave F..."
1,Jajah,mobile,$33M,"{'description': 'Jajah HQ', 'address1': '2513 ..."
2,blogTV,games_video,$3M,"{'description': 'Headquarters', 'address1': '3..."
3,Pando Networks,games_video,$11M,"{'description': None, 'address1': '520 Broadwa..."
4,AdaptiveBlue,games_video,$24M,"{'description': '', 'address1': '131 Varick St..."
...,...,...,...,...
638,smartFOCUS,software,$30M,"{'description': 'Hong Kong', 'address1': '703 ..."
639,PaymentOne,ecommerce,$7M,"{'description': '', 'address1': '5883 Rue Ferr..."
640,Vidavee,network_hosting,$6M,NaN
641,Layered Technologies,network_hosting,$63M,"{'description': 'Headquarters', 'address1': '5..."


In [6]:
#Imported function for lib.py. Transforming office object into GeoPoint for office
techdata_clean = techdata.apply(OFFToGeoPoint,axis=1, result_type="expand")
techdata_clean.columns = ["office","clean_state"]
techcompany_processed = pd.concat([techdata,techdata_clean], axis=1)
techcompany_processed = techcompany_processed[["name","category_code","office","total_money_raised","clean_state"]]
techcompany_processed

,name,category_code,office,total_money_raised,clean_state
0,Joost,games_video,"{'type': 'Point', 'coordinates': [-74.0094471,...",$45M,success
1,Jajah,mobile,"{'type': 'Point', 'coordinates': [-122.0899512...",$33M,success
2,blogTV,games_video,"{'type': 'Point', 'coordinates': [34.851612, 3...",$3M,success
3,Pando Networks,games_video,"{'type': 'Point', 'coordinates': [-73.99873, 4...",$11M,success
4,AdaptiveBlue,games_video,"{'type': 'Point', 'coordinates': [-74.3372, 40...",$24M,success
...,...,...,...,...,...
638,smartFOCUS,software,None,$30M,Invalid lat lat and long
639,PaymentOne,ecommerce,"{'type': 'Point', 'coordinates': [-121.776935,...",$7M,success
640,Vidavee,network_hosting,None,$6M,No office
641,Layered Technologies,network_hosting,None,$63M,Invalid lat lat and long


In [7]:
#Cleaning NA
techcompany_processed=techcompany_processed.dropna()
techcompany_processed

,name,category_code,office,total_money_raised,clean_state
0,Joost,games_video,"{'type': 'Point', 'coordinates': [-74.0094471,...",$45M,success
1,Jajah,mobile,"{'type': 'Point', 'coordinates': [-122.0899512...",$33M,success
2,blogTV,games_video,"{'type': 'Point', 'coordinates': [34.851612, 3...",$3M,success
3,Pando Networks,games_video,"{'type': 'Point', 'coordinates': [-73.99873, 4...",$11M,success
4,AdaptiveBlue,games_video,"{'type': 'Point', 'coordinates': [-74.3372, 40...",$24M,success
...,...,...,...,...,...
631,GoTV Networks,mobile,"{'type': 'Point', 'coordinates': [-118.448703,...",$28M,success
632,Venturi Wireless,mobile,"{'type': 'Point', 'coordinates': [-122.0018725...",$10M,success
633,FinAnalytica,software,"{'type': 'Point', 'coordinates': [-73.978887, ...",$7M,success
638,smartFOCUS,software,"{'type': 'Point', 'coordinates': [-2.5909744, ...",$30M,success


In [8]:
techcompany_location=pd.concat([techcompany_processed, techcompany_processed.apply(easyLatLng, axis=1, result_type="expand")], axis=1)
techcompany_location

,name,category_code,office,total_money_raised,clean_state,longitude,latitude
0,Joost,games_video,"{'type': 'Point', 'coordinates': [-74.0094471,...",$45M,success,-74.009447,40.746497
1,Jajah,mobile,"{'type': 'Point', 'coordinates': [-122.0899512...",$33M,success,-122.089951,37.423390
2,blogTV,games_video,"{'type': 'Point', 'coordinates': [34.851612, 3...",$3M,success,34.851612,31.046051
3,Pando Networks,games_video,"{'type': 'Point', 'coordinates': [-73.99873, 4...",$11M,success,-73.998730,40.722655
4,AdaptiveBlue,games_video,"{'type': 'Point', 'coordinates': [-74.3372, 40...",$24M,success,-74.337200,40.801358
...,...,...,...,...,...,...,...
631,GoTV Networks,mobile,"{'type': 'Point', 'coordinates': [-118.448703,...",$28M,success,-118.448703,34.150840
632,Venturi Wireless,mobile,"{'type': 'Point', 'coordinates': [-122.0018725...",$10M,success,-122.001873,37.409999
633,FinAnalytica,software,"{'type': 'Point', 'coordinates': [-73.978887, ...",$7M,success,-73.978887,40.751293
638,smartFOCUS,software,"{'type': 'Point', 'coordinates': [-2.5909744, ...",$30M,success,-2.590974,51.452512


In [9]:
#company_location.to_json("query_ts.json",orient="records").I just have checked where the offices are located in a map

# Fulfilling designers´s preferens

In [10]:
design_query= {"$or":[{"category_code":"design"},{"category_code":"web"}]}

data_design=db.companies.find(design_query,{"offices":1,"name": 1,"category_code":1,"_id":0})
data_design=list(data_design)
len(data_design)

3791

In [11]:
#As a DataFrame
designdata = pd.DataFrame(data_design)
designdata = designdata.explode("offices")
designdata

,name,category_code,offices
0,Wetpaint,web,"{'description': '', 'address1': '710 - 2nd Ave..."
0,Wetpaint,web,"{'description': '', 'address1': '270 Lafayette..."
1,Postini,web,"{'description': None, 'address1': '959 Skyway ..."
2,Geni,web,"{'description': 'Headquarters', 'address1': '9..."
3,StumbleUpon,web,"{'description': '', 'address1': '', 'address2'..."
...,...,...,...
3786,Optemo,web,"{'description': '', 'address1': '246 - 970 Bur..."
3787,Zoomission,web,NaN
3788,Eazeeloans,web,"{'description': 'Apex Finance & Marketing', 'a..."
3789,DocASAP,web,"{'description': '', 'address1': '115 5th Ave',..."


In [12]:
#Imported function for lib.py. Transforming office object into GeoPoint for office
designdata_clean = designdata.apply(OFFToGeoPoint,axis=1, result_type="expand")
designdata_clean.columns = ["office","clean_state"]
designcompany_processed = pd.concat([designdata,designdata_clean], axis=1)
designcompany_processed = designcompany_processed[["name","category_code","office","clean_state"]]
designcompany_processed

,name,category_code,office,clean_state
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-122.333253,...",success
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-73.9964312,...",success
1,Postini,web,"{'type': 'Point', 'coordinates': [-122.247573,...",success
2,Geni,web,"{'type': 'Point', 'coordinates': [-118.393064,...",success
3,StumbleUpon,web,"{'type': 'Point', 'coordinates': [-122.419204,...",success
...,...,...,...,...
3786,Optemo,web,None,Invalid lat lat and long
3787,Zoomission,web,None,No office
3788,Eazeeloans,web,None,Invalid lat lat and long
3789,DocASAP,web,None,Invalid lat lat and long


In [13]:
#Cleaning NA
designcompany_processed=designcompany_processed.dropna()
designcompany_processed

,name,category_code,office,clean_state
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-122.333253,...",success
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-73.9964312,...",success
1,Postini,web,"{'type': 'Point', 'coordinates': [-122.247573,...",success
2,Geni,web,"{'type': 'Point', 'coordinates': [-118.393064,...",success
3,StumbleUpon,web,"{'type': 'Point', 'coordinates': [-122.419204,...",success
...,...,...,...,...
3769,NameJet,web,"{'type': 'Point', 'coordinates': [-122.1295044...",success
3770,eNom,web,"{'type': 'Point', 'coordinates': [-122.1295044...",success
3774,Conversition,web,"{'type': 'Point', 'coordinates': [-73.986951, ...",success
3779,Gastrogate,web,"{'type': 'Point', 'coordinates': [16.5595393, ...",success


In [449]:
designcompany_location=pd.concat([designcompany_processed, designcompany_processed.apply(easyLatLng, axis=1, result_type="expand")], axis=1)
designcompany_location=designcompany_location.rename(columns={"longitude": "longitude2", "latitude": "latitude2"}) #to be able to apply the next function
designcompany_location

,name,category_code,office,clean_state,longitude2,latitude2
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-122.333253,...",success,-122.333253,47.603122
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-73.9964312,...",success,-73.996431,40.723731
1,Postini,web,"{'type': 'Point', 'coordinates': [-122.247573,...",success,-122.247573,37.506885
2,Geni,web,"{'type': 'Point', 'coordinates': [-118.393064,...",success,-118.393064,34.090368
3,StumbleUpon,web,"{'type': 'Point', 'coordinates': [-122.419204,...",success,-122.419204,37.775196
...,...,...,...,...,...,...
3769,NameJet,web,"{'type': 'Point', 'coordinates': [-122.1295044...",success,-122.129504,47.631518
3770,eNom,web,"{'type': 'Point', 'coordinates': [-122.1295044...",success,-122.129504,47.631518
3774,Conversition,web,"{'type': 'Point', 'coordinates': [-73.986951, ...",success,-73.986951,40.756054
3779,Gastrogate,web,"{'type': 'Point', 'coordinates': [16.5595393, ...",success,16.559539,59.617364


##### I am going to calculate the distance between the companies of this two categories

In [428]:
designcompanyMerge = designcompany_location.reset_index(drop=True)
techcompanyMerge = techcompany_location.reset_index(drop=True)

total_companies = pd.concat([techcompanyMerge,designcompanyMerge], axis=1, sort=False)
total_companies['Distance'] = total_companies.apply(lambda x: distance(x['latitude'], x['longitude'], x['latitude2'], x['longitude2']), axis=1)
total_companies = total_companies.sort_values(by='Distance', ascending=True).reset_index(drop=True)
total_companies.head(3)

,name,category_code,office,total_money_raised,clean_state,longitude,latitude,name,category_code,office,clean_state,longitude2,latitude2,Distance
0,bluepulse,network_hosting,"{'type': 'Point', 'coordinates': [-122.324688,...",$6M,success,-122.324688,37.563905,Respectance,web,"{'type': 'Point', 'coordinates': [-122.321836,...",success,-122.321836,37.559090,0.591669
1,Jumptap,mobile,"{'type': 'Point', 'coordinates': [-73.980305, ...",$122M,success,-73.980305,40.751478,Mimeo,web,"{'type': 'Point', 'coordinates': [-73.982807, ...",success,-73.982807,40.745216,0.727731
2,OpSource,network_hosting,"{'type': 'Point', 'coordinates': [-121.976473,...",$49M,success,-121.976473,37.406217,Vobile,web,"{'type': 'Point', 'coordinates': [-121.97895, ...",success,-121.978950,37.399208,0.809749


**With this distance information and with the location of the companies in a map I did before, my first possible city is "Palo Alto" in California, USA (37.414289,-122.098712). I am going to check now if this location can fulfill requirements of the other workers.**
 

Once I know the companies I want to locate in the map, I am going to prepare the date for Geolocation

In [523]:
#Select the company choosed from design
FirstPreferenced=designcompany_location[(designcompany_location['name']=="Vobile")] 

#Select the company choosed from tech
FirstPreferencet=techcompany_location[(techcompany_location['name']=="OpSource")]

#Change names to normalize both df
FirstPreferenced=FirstPreferenced.rename(columns={"longitude2": "longitude", "latitude2": "latitude"})

#Concat df
FirstPreference= pd.concat([FirstPreferenced,FirstPreferencet])

#Dropping not interesting columns and change name
FirstPreference = FirstPreference.drop(['clean_state', 'total_money_raised', "category_code"], axis = 'columns')
FirstPreference=FirstPreference.rename(columns={"office": "coord"})
FirstPreference

,name,coord,longitude,latitude
365,Vobile,"{'type': 'Point', 'coordinates': [-121.97895, ...",-121.978950,37.399208
360,OpSource,"{'type': 'Point', 'coordinates': [-121.976473,...",-121.976473,37.406217


# Fulfilling Kindergarten , pubs and Starbucks needs

**KINDERGARTEN**

In [231]:
#Imported from lib.py.  

child_options=getFromFS("37.414289,-122.098712","5000","Child Daycare") #Match!
#child_options. I just comment the output because it is too long.                         

200


In [233]:
#Imported function from lib.py.

child=FS_to_df(child_options)
child

,nombre,long,lat,Distance
0,Peninsula Day Care Center,37.413807,-122.104683,0.53


**PUBS**

In [223]:
pub_options=getFromFS("37.414289,-122.098712","1000","pub")#Match!
#pub_options

200


In [232]:
pub=FS_to_df(pub_options)
pub

,nombre,long,lat,Distance
0,O’Malley’s Sports Pub,37.414268,-122.093625,0.449


**STARBUCKS**

In [228]:
star_options=getFromFS("37.414289,-122.098712","2000" ,"starbucks")#Match!
#star_options

200


In [234]:
starbucks=FS_to_df(star_options)
starbucks

,nombre,long,lat,Distance
0,Starbucks,37.411454,-122.094273,0.503


Once I know the locals I want to locate them in the map, I am going to prepare the date for Geolocation

In [522]:
#Concat the three variables df
AllFSrequest=pd.concat([pub,child,starbucks])

#Rename some columns
AllFSrequest=AllFSrequest.rename(columns={"long": "longitude", "lat": "latitude","nombre":"name"})

#Reset index
AllFSrequest=AllFSrequest.reset_index(drop=True)

#Create a column with dict type
f=lambda x,y:dict({"Type":"Point","coordinates":[x,y]})
AllFSrequest['coord'] = AllFSrequest.apply(lambda x:f(x.latitude, x.longitude), axis=1)

#Delete some columns
AllFSrequest=AllFSrequest.drop(["Distance"], axis = 'columns')

AllFSrequest


,name,longitude,latitude,coord
0,O’Malley’s Sports Pub,37.414268,-122.093625,"{'Type': 'Point', 'coordinates': [-122.0936247..."
1,Peninsula Day Care Center,37.413807,-122.104683,"{'Type': 'Point', 'coordinates': [-122.1046829..."
2,Starbucks,37.411454,-122.094273,"{'Type': 'Point', 'coordinates': [-122.0942725..."


It seems that I have found the perfect location for the office but I would like to be ensured that there is an airport close to it.

## AIRPORT NEED 

In [235]:
#From https://datahub.io/core/airport-codes#data downloaded the JSON

In [350]:
airports = list(db.airports.find({}))
df_airports= pd.DataFrame(airports)
df_airports.head(3)

,_id,name,city,country,iata_code,_geoloc,links_count,objectID
0,5e9dace1e3f88022ea8a96da,Hartsfield Jackson Atlanta Intl,Atlanta,United States,ATL,"{'lat': 33.636719, 'lng': -84.428067}",1826,3682
1,5e9dace1e3f88022ea8a96db,Chicago Ohare Intl,Chicago,United States,ORD,"{'lat': 41.978603, 'lng': -87.904842}",1108,3830
2,5e9dace1e3f88022ea8a96dc,Capital Intl,Beijing,China,PEK,"{'lat': 40.080111, 'lng': 116.584556}",1069,3364


In [361]:
#Importing GEOToGeoPoint from lib.py

airports_geo = df.apply(GEOToGeoPoint,axis=1, result_type="expand")
airports_geo.head(2)

,0,1
0,"{'type': 'Point', 'coordinates': [-84.428067, ...",success
1,"{'type': 'Point', 'coordinates': [-87.904842, ...",success


In [372]:
airports_processed = pd.concat([df_airports,airports_geo], axis=1)
airports_processed.head(3)

,_id,name,city,country,iata_code,_geoloc,links_count,objectID,0,1
0,5e9dace1e3f88022ea8a96da,Hartsfield Jackson Atlanta Intl,Atlanta,United States,ATL,"{'lat': 33.636719, 'lng': -84.428067}",1826,3682,"{'type': 'Point', 'coordinates': [-84.428067, ...",success
1,5e9dace1e3f88022ea8a96db,Chicago Ohare Intl,Chicago,United States,ORD,"{'lat': 41.978603, 'lng': -87.904842}",1108,3830,"{'type': 'Point', 'coordinates': [-87.904842, ...",success
2,5e9dace1e3f88022ea8a96dc,Capital Intl,Beijing,China,PEK,"{'lat': 40.080111, 'lng': 116.584556}",1069,3364,"{'type': 'Point', 'coordinates': [116.584556, ...",success


In [420]:
#Cleaning NA
airports_clean=airports_processed.dropna()
airports_clean=airports_clean.rename(columns={0:"coord",1:"status"})
airports_clean.head(3)

,_id,name,city,country,iata_code,_geoloc,links_count,objectID,coord,status
0,5e9dace1e3f88022ea8a96da,Hartsfield Jackson Atlanta Intl,Atlanta,United States,ATL,"{'lat': 33.636719, 'lng': -84.428067}",1826,3682,"{'type': 'Point', 'coordinates': [-84.428067, ...",success
1,5e9dace1e3f88022ea8a96db,Chicago Ohare Intl,Chicago,United States,ORD,"{'lat': 41.978603, 'lng': -87.904842}",1108,3830,"{'type': 'Point', 'coordinates': [-87.904842, ...",success
2,5e9dace1e3f88022ea8a96dc,Capital Intl,Beijing,China,PEK,"{'lat': 40.080111, 'lng': 116.584556}",1069,3364,"{'type': 'Point', 'coordinates': [116.584556, ...",success


In [421]:
airports_clean.to_json("airports_clean.json",orient="records",default_handler=str)#Export asjson to do mongodb import

In [560]:
query_PA= { "coord" :{
                "$near":{
                    "$geometry": {"type":"Point","coordinates": [-122.098712, 37.414289] },
                    "$minDistance": 1000,
                    "$maxDistance": 20000
                   }
          }}
airport_PA = list(db.GEOairports.find(query_PA,{"name":1,"coord":1,"_id":0}))
PA=list(airport_PA)
print(PA)

[{'name': 'Norman Y Mineta San Jose Intl', 'coord': {'type': 'Point', 'coordinates': [-121.929022, 37.3626]}}]


In [561]:
PA=pd.DataFrame(PA)
PA

,name,coord
0,Norman Y Mineta San Jose Intl,"{'type': 'Point', 'coordinates': [-121.929022,..."


In [562]:
#Import function from lib.py

PA_coor=PA.apply(easyLatLng, axis=1, result_type="expand")
PA_coor

,latitude,longitude
0,37.3626,-121.929022


In [563]:
PaloAlto_airport=pd.concat([PA, PA.apply(easyLatLng, axis=1, result_type="expand")], axis=1)
PaloAlto_airport

,name,coord,latitude,longitude
0,Norman Y Mineta San Jose Intl,"{'type': 'Point', 'coordinates': [-121.929022,...",37.3626,-121.929022


### DATAFRAME RESUME ALL INFO

In [564]:
all_info=pd.concat([FirstPreference,PaloAlto_airport,AllFSrequest], axis=0)
all_info=all_info.reset_index(drop=True)
all_info

,name,coord,longitude,latitude
0,Vobile,"{'type': 'Point', 'coordinates': [-121.97895, ...",-121.978950,37.399208
1,OpSource,"{'type': 'Point', 'coordinates': [-121.976473,...",-121.976473,37.406217
2,Norman Y Mineta San Jose Intl,"{'type': 'Point', 'coordinates': [-121.929022,...",-121.929022,37.362600
3,O’Malley’s Sports Pub,"{'Type': 'Point', 'coordinates': [-122.0936247...",37.414268,-122.093625
4,Peninsula Day Care Center,"{'Type': 'Point', 'coordinates': [-122.1046829...",37.413807,-122.104683
5,Starbucks,"{'Type': 'Point', 'coordinates': [-122.0942725...",37.411454,-122.094273


In [565]:
all_info.to_json("all_info.json",orient="records") #To geolocate